In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

    # 心血管疾患による死亡者の予測

心血管疾患(CVDs)は世界で最も多い死因であり，世界中で1年間におよそ1790万人の人が亡くなっているといわれている．これは世界中の死者の約31 %を占めている．

# 0.特徴量の説明

- 目的変数  
**DEATH_EVENT** : 死亡したかどうか(0=生存, 1=死亡)  
<br>
- 説明変数  
**age** : 年齢  
**anaemia** : 貧血(赤血球やヘモグロビンの減少)か否か(0=貧血でない, 1=貧血)  
**creatinine_phosphokinase** : 血液中のCPK(クレアチンホスキナーゼ)の量( mcg/L)  
(注)クレアチンホスキナーゼ: 動物が持つ酵素であり，筋肉の収縮の際のエネルギー代謝に関与している．   
**diabetes** : 糖尿病か否か(0=糖尿病でない, 1=糖尿病)  
**ejection_fraction** : 駆出率(1回の心臓の収縮で血液を送り出す血液量( %))  
**high_blood_pressure** : 高血圧か否か(0=高血圧でない, 1=高血圧)  
**platelets** : 血液中の血小板の量( kplatelets/mL)  
**serum_creatinine** :  血液中の血清クレアチニンの量( mg/dL)  
(注)クレアチニン:筋肉に含まれるタンパク質の老廃物．  
**serum_sodium** : 血液中の血清ナトリウムの量( mEq/L)  
**sex** : 性別(0=女性, 1=男性)  
**smoking** : 喫煙者か否か(0=喫煙者でない, 1=喫煙者)  
**time** : 追跡調査期間( 日数)  

In [ ]:
# 1.モジュールのインポート

In [ ]:
# モジュールのインポート
# 基本モジュールのインポート
import numpy as np
from numpy import nan
import pandas as pd
from pandas import DataFrame,Series

# 可視化のモジュールのインポート
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import matplotlib.colors as mcolors
import matplotlib as mpl

# 不要な警告メッセージを出さない
import warnings
warnings.filterwarnings('ignore')

# 標準化データを作る
from sklearn.preprocessing import StandardScaler


# 目的変数が与えられているデータを'train'と'test'データに分けるためのモジュール
from sklearn.model_selection import train_test_split

# 学習モデルのためのモジュール
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

# 主成分分析のためのモジュール
from sklearn.decomposition import PCA

# 評価のためのモジュール
from sklearn import metrics

In [ ]:
# 3D散布図のためのモジュールのインポート
from mpl_toolkits.mplot3d import Axes3D

# 2.データの読み込み

In [ ]:
# データの読み込み
df = pd.read_csv('../input/heart-failure-clinical-data/heart_failure_clinical_records_dataset.csv')

# 3.データの概観

In [ ]:
# データの特徴量一覧
df.columns.values

In [ ]:
# データの確認
df.head()

In [ ]:
df.tail()

In [ ]:
# データの概観
print(df.info())

In [ ]:
# 欠損値の確認
df.isnull().sum()

欠損値はなし

In [ ]:
df.describe(include='all')

In [ ]:
# 主成分分析用のデータ
# 目的変数のデータ
df_target = df[['DEATH_EVENT']] 
# 説明変数のデータ
df_data = df.drop('DEATH_EVENT', axis=1)

# 4.データの分析(相関関係などを見ていく)

**4.1. 相関関係を見る**

In [ ]:
# 散布図行列を出力する
sns.pairplot(df, size=2.0, hue='DEATH_EVENT', markers='+')
plt.show()

In [ ]:
# 強い関係がありそうな特徴量だけを用いた散布図行列
# 新たにDataFrameを作る
df_pickup = df[['age', 'creatinine_phosphokinase', 'ejection_fraction', 'time', 'sex','serum_creatinine', 'serum_sodium', 'DEATH_EVENT']]
# 散布図行列を作る
sns.pairplot(df_pickup, size=2.0, hue='DEATH_EVENT', markers='+')
plt.show()

In [ ]:
# 相関係数行列
corr_mat = df.corr(method='pearson')
corr_mat

In [ ]:
# 相関係数行列を可視化
sns.heatmap(corr_mat,
           vmin=-1.0,
           vmax=1.0,
           center=0,
           annot=True,
           fmt='.1f',
           xticklabels=corr_mat.columns.values,
           yticklabels=corr_mat.columns.values
           )
plt.show()

**4.2.各説明変数について見ていく**

- 'age'

In [ ]:
# 'age'について
# 生死別の年齢分布を見る
g = sns.FacetGrid(df, col='DEATH_EVENT')
g.map(plt.hist, 'age', bins=20) 

In [ ]:
# 全サンプルの年代分布
g = sns.FacetGrid(df)
g.map(plt.hist, 'age', bins=20)

In [ ]:
# 年代毎に死亡率を見る
# 5歳毎のレンジで区切る
df['agegroup_5age'] = (df['age'] // 5) * 5
df.head()

In [ ]:
# 'agegroup_5age'でグループ化して死亡率を見る
group_5age = df[['agegroup_5age', 'DEATH_EVENT']].groupby('agegroup_5age', as_index=False)
df_agegroup = group_5age.mean().sort_values(by='DEATH_EVENT', ascending=False)
# それぞれの年代に何人いるか
df_agegroup['count'] = group_5age.count().DEATH_EVENT
df_agegroup

- 45～49 歳以外は年齢が上がるほど死亡率が高くなる
- 40～45 歳は死亡率が低い

- 'anaemia'(貧血)

In [ ]:
# 'anaemia'かどうかで死亡率を見る
group_anaemia = df[['anaemia', 'DEATH_EVENT']].groupby(by='anaemia', as_index=False)
group_anaemia.mean().sort_values(by='DEATH_EVENT', ascending=False)

'anaemia'だと死亡率が5 %ほど上がる

- 'creatinine_phosphokinase'

In [ ]:
# 'creatinine_phosphokinase'の値の分布
g = sns.FacetGrid(df, col='DEATH_EVENT')
g.map(plt.hist, 'creatinine_phosphokinase', bins=20)

値が6000を超えるとほぼ確実に死亡している

- 'diabetes'(糖尿病)

In [ ]:
# 'diabetes'かどうかで死亡率を見る
group_diabetes = df[['diabetes', 'DEATH_EVENT']].groupby(by='diabetes', as_index=False)
group_diabetes.mean().sort_values(by='DEATH_EVENT', ascending=False)

'diabetes'だけでは死亡率に大きく関わらない

- 'ejection_fraction'(駆出率)

In [ ]:
# 'ejection_fraction'の分布を見る
g = sns.FacetGrid(df, col='DEATH_EVENT')
g.map(plt.hist, 'ejection_fraction', bins=20)

In [ ]:
# 'ejection_fraction'の値ごとの死亡率
# 値を5刻みで分ける
df['ejection_fraction_range'] = (df['ejection_fraction'] // 5) *5

# 'ejection_fraction_range'でグループ化
group_EF_range = df[['ejection_fraction_range', 'DEATH_EVENT']].groupby(by='ejection_fraction_range', as_index=False)
df_EF = group_EF_range.mean().sort_values(by='DEATH_EVENT', ascending=False)
df_EF

In [ ]:
# 死亡率を可視化
plt.scatter(df_EF.ejection_fraction_range, df_EF.DEATH_EVENT)

# 軸の設定
plt.xlabel('ejection fraction (%)')
plt.ylabel('death rate')

plt.show()

- 40～60は正常である．
- 低すぎたり高すぎると死亡率が上がる

In [ ]:
# 生存している人の平均値からどれくらい離れているかを説明変数として取り入れる
# 生存者の'ejection_fraction'の平均値
EF_mean = df.loc[df['DEATH_EVENT'] == 0, 'ejection_fraction'].mean()
print('生存者のejection_fractionの平均値:{:.3f}'.format(EF_mean))

In [ ]:
# 生存者の平均値からどのくらい離れているか
df['ejection_fraction_dist'] = np.absolute(df.ejection_fraction - EF_mean)

In [ ]:
# 'ejection_fraction_dist'のヒストグラム
g = sns.FacetGrid(df, col='DEATH_EVENT')
g.map(plt.hist, 'ejection_fraction_dist', bins=15)

In [ ]:
# 'ejection_fraction_dist'の値を10ごとにグループ化
# 'ejection_fraction_dist'の値を10ごとに刻む
df['ejection_fraction_dist'] = (df['ejection_fraction_dist'] // 10) * 10

# 'ejection_fraction_dist'をグループ化
group_ejection_fraction_dist = df[['ejection_fraction_dist', 'DEATH_EVENT']].groupby(by='ejection_fraction_dist', as_index=False)
df_EFdist = group_ejection_fraction_dist.mean().sort_values(by='DEATH_EVENT', ascending=False)
df_EFdist['count'] = group_ejection_fraction_dist.count().DEATH_EVENT
df_EFdist

'ejection_fraction_dist'が大きいほど死亡率が高くなる

'ejection_fraction_dist'が大きいほど死亡率が高くなる

In [ ]:
# 高血圧持ちかどうかで死亡率が変わるかどうか
group_pressure = df[['high_blood_pressure', 'DEATH_EVENT']].groupby(by='high_blood_pressure', as_index=False)
group_pressure.mean().sort_values(by='DEATH_EVENT', ascending=False)

高血圧持ちだと死亡率がおよそ7 %上がる

- 'platelets'(血液中の血小板の量)

In [ ]:
# 'platelets'のヒストグラム
g = sns.FacetGrid(df, col='DEATH_EVENT')
g.map(plt.hist, 'platelets', bins=20)

In [ ]:
# 'platelets'を50000刻みで分ける
df['platelets_50k'] = (df['platelets'] // 50000)*50000

# 'platelets_50k'をグループ化
group_platelets = df[['platelets_50k', 'DEATH_EVENT']].groupby(by='platelets_50k', as_index=False)
df_platelets = group_platelets.mean().sort_values(by='DEATH_EVENT', ascending=False)
df_platelets['count'] = group_platelets.count().DEATH_EVENT
df_platelets

In [ ]:
# 'platelets'と死亡率の関係を可視化
# グラフをプロットする
plt.scatter(df_platelets.platelets_50k, df_platelets.DEATH_EVENT)

# 軸を設定
plt.xlabel('platelets range (kplatelets/mL)')
plt.ylabel('death ratio')

plt.show()

  600000以上の値は標本が1つしかないことを考慮すれば，血小板の量単体では死亡率にあまり関係しない

In [ ]:
# 生存している人の平均値からどれくらい離れているかを説明変数として取り入れる
# 生存者の'platelets'の平均値
pl_mean = df.loc[df['DEATH_EVENT'] == 0, 'platelets'].mean()
print('生存者のplateletsの平均値:{:.3f}'.format(pl_mean))

In [ ]:
# 生存者の平均値からどのくらい離れているか
df['platelets_dist'] = np.absolute(df.platelets - pl_mean)

In [ ]:
# 'platelets_dist'のヒストグラム
g = sns.FacetGrid(df, col='DEATH_EVENT')
g.map(plt.hist, 'platelets_dist', bins=15)

In [ ]:
# 'platelets_dist'の値を40000ごとにグループ化
# 'platelets_dist'の値を40000ごとに刻む
df['platelets_dist'] = (df['platelets_dist'] // 40000) * 40000

# 'platelets_dist'をグループ化
group_platelets_dist = df[['platelets_dist', 'DEATH_EVENT']].groupby(by='platelets_dist', as_index=False)
df_pldist = group_platelets_dist.mean().sort_values(by='DEATH_EVENT', ascending=False)
df_pldist['count'] = group_platelets_dist.count().DEATH_EVENT
df_pldist

In [ ]:
# 'platelets'と死亡率の関係を可視化
# グラフをプロットする
plt.scatter(df_pldist.platelets_dist, df_pldist.DEATH_EVENT)

# 軸を設定
plt.xlabel('difference from the mean)')
plt.ylabel('death ratio')

plt.show()

- 'serum_creatinine'(血液中の血清クレアチニンの量( mg/dL))

In [ ]:
# 'serum_creatinine'のヒストグラム
g = sns.FacetGrid(df, col='DEATH_EVENT')
g.map(plt.hist, 'serum_creatinine', bins=10)

In [ ]:
# serum_creatinineの各範囲の値における死亡率
# serum_creatinineを2刻みで分ける
df['serum_creatinine_2'] = (df['serum_creatinine'] // 2) * 2
# 'serum_creatinine_2'をグループ化
group_serum_creatinine = df[['serum_creatinine_2', 'DEATH_EVENT']].groupby(by='serum_creatinine_2', as_index=False)
df_SC = group_serum_creatinine.mean().sort_values(by='DEATH_EVENT', ascending=False)
df_SC['count'] = group_serum_creatinine.count().DEATH_EVENT
df_SC

In [ ]:
# 'serum_creatinine_2'の各値ごとの死亡率を可視化
plt.scatter(df_SC.serum_creatinine_2, df_SC.DEATH_EVENT)

# 軸名を設定
plt.xlabel('serum creatinine (mg/dL)')
plt.ylabel('deth ratio')

plt.show()

- 'serum_sodium'(血液中の血清ナトリウムの量( mEq/L))

In [ ]:
# 'serum_sodium'のヒストグラム
g = sns.FacetGrid(df, col='DEATH_EVENT')
g.map(plt.hist, 'serum_sodium', bins=20)

In [ ]:
# 'serum_sodium'を10刻みで分ける
df['serum_sodium_10'] = (df['serum_sodium'] // 10) * 10
# 'serum_sodium_10'をグループ化
group_serum_sodium = df[['serum_sodium_10', 'DEATH_EVENT']].groupby(by='serum_sodium_10', as_index=False)
df_SS = group_serum_sodium.mean().sort_values(by='DEATH_EVENT', ascending=False)
df_SS['count'] = group_serum_sodium.count().DEATH_EVENT
df_SS

In [ ]:
# 'serum_sodium'と死亡率の関係を可視化する
plt.scatter(df_SS.serum_sodium_10, df_SS.DEATH_EVENT)

# 軸の設定
plt.xlabel('serum sodium (mEq/L)')
plt.ylabel('death ratio')

plt.show()

- 130よりも低いと死亡率が高い
- 130以上では，値が高いほど死亡率が低い

In [ ]:
# 生存している人の平均値からどれくらい離れているかを説明変数として取り入れる
# 生存者の'serum_sodium'の平均値
SS_mean = df.loc[df['DEATH_EVENT'] == 0, 'serum_sodium'].mean()
print('生存者のserum_sodiumの平均値:{:.3f}'.format(SS_mean))

In [ ]:
# 生存者の平均値からどのくらい離れているか
df['serum_sodium_dist'] = np.absolute(df.serum_sodium - SS_mean)

In [ ]:
# 'platelets_dist'のヒストグラム
g = sns.FacetGrid(df, col='DEATH_EVENT')
g.map(plt.hist, 'serum_sodium_dist', bins=15)

In [ ]:
# 'serum_sodium_dist'の値を40000ごとにグループ化
# 'serum_sodium_dist'の値を40000ごとに刻む
df['serum_sodium_dist'] = (df['serum_sodium_dist'] // 1) * 1

# 'serum_sodium_dist'をグループ化
group_serum_sodium_dist = df[['serum_sodium_dist', 'DEATH_EVENT']].groupby(by='serum_sodium_dist', as_index=False)
df_SSdist = group_serum_sodium_dist.mean().sort_values(by='DEATH_EVENT', ascending=False)
df_SSdist['count'] = group_serum_sodium_dist.count().DEATH_EVENT
df_SSdist

In [ ]:
# 'serum_sodium_dist'の死亡率を可視化
# グラフをプロットする
plt.scatter(df_SSdist.serum_sodium_dist, df_SSdist.DEATH_EVENT)

# 軸を設定
plt.xlabel('difference from the mean')
plt.ylabel('death ratio')

plt.show()

- 'sex'(性別(0=女性, 1=男性))

In [ ]:
# 性別ごとの死亡率
group_sex = df[['sex', 'DEATH_EVENT']].groupby(by='sex', as_index=False)
group_sex.mean().sort_values(by='DEATH_EVENT', ascending=False)

性別はほぼ関係しないといえる

- 'smoking'(喫煙者か否か(0=喫煙者でない, 1=喫煙者))

In [ ]:
# 喫煙が与える影響
group_smoking = df[['smoking', 'DEATH_EVENT']].groupby(by='smoking', as_index=False)
group_smoking.mean().sort_values(by='DEATH_EVENT', ascending=False)

喫煙は死亡率にほぼ関係しないといえる

- 'time' 追跡調査期間( 日数)

In [ ]:
# 'time'のヒストグラム
g = sns.FacetGrid(df, col='DEATH_EVENT')
g.map(plt.hist, 'time', bins=20)

In [ ]:
# 'time'を25刻みで分ける
df['time_25'] = (df['time'] // 25 ) * 25
group_time = df[['time_25', 'DEATH_EVENT']].groupby(by='time_25', as_index=False)
df_time = group_time.mean().sort_values(by='DEATH_EVENT', ascending=False)
df_time['count'] = group_time.count().DEATH_EVENT
df_time

In [ ]:
# 'time'と死亡率の関係の可視化
plt.scatter(df_time.time_25, df_time.DEATH_EVENT)

# 軸の設定
plt.xlabel('time (days)')
plt.ylabel('death ratio')

plt.show()

50日以内だと死亡率が高い

**4.2.新たな説明変数の作成**

In [ ]:
# 持病の数と死亡率の関係
# 持病の数の列を作成
df['number of chronic condition'] = df.anaemia + df.diabetes + df.high_blood_pressure + df.smoking

In [ ]:
group_chronic = df[['number of chronic condition', 'DEATH_EVENT']].groupby(by='number of chronic condition', as_index=False)
df_chronic = group_chronic.mean().sort_values(by='DEATH_EVENT', ascending=False)
df_chronic['count'] = group_chronic.count().DEATH_EVENT
df_chronic

単純な持病の数の合計には死亡率と相関関係がない

In [ ]:
# 指定した持病の数と死亡率の関係
# 指定した持病の数の列を作成
df['risk factor'] = (df.diabetes ) + (df.high_blood_pressure * 2) + (df.smoking)

In [ ]:
# 相関係数行列を出力する
corr_mat2 = df.corr(method='pearson')
corr_mat2[['DEATH_EVENT']]

In [ ]:
# 'df'のcolumnsの一覧
df.columns.values

# 5.データの整理と加工

In [ ]:
# 'df'の確認
df.head()

In [ ]:
# 不要な列の削除
df_process = df.drop(['age', 'creatinine_phosphokinase', 'ejection_fraction', 'platelets', 'serum_creatinine', 'serum_sodium', 'time', 'number of chronic condition', 'ejection_fraction_range', 'platelets_50k', 'serum_sodium_10', 'diabetes', 'smoking', 'high_blood_pressure'], axis=1)

In [ ]:
df_process.head()

In [ ]:
# 相関係数行列を出力する
corr_mat_process = df_process.corr(method='pearson')
corr_mat_process

In [ ]:
# 相関行列を可視化する
sns.heatmap(corr_mat_process,
           vmin=-1.0,
           vmax=1.0,
           center=0,
           annot=True,
           fmt='.1f',
           xticklabels=corr_mat_process.columns.values,
           yticklabels=corr_mat_process.columns.values
           )
plt.show()

In [ ]:
# 説明変数と目的変数を分ける
# 目的変数
Y = df[['DEATH_EVENT']]
# 説明変数
X = df.drop('DEATH_EVENT', axis=1)

In [ ]:
# 'train'データと'test'データを分ける
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, random_state=0)

# 6.学習とテスト

**6.1.各モデルによる予測**

- ロジスティック回帰

In [ ]:
# モデルのインスタンスの作成
log_model = LogisticRegression(max_iter=1000)

# 学習
log_model.fit(X_train, Y_train)

In [ ]:
# 'test'データの予測
log_predict = log_model.predict(X_test)

# 予測の評価
log_accuracy = metrics.accuracy_score(Y_test, log_predict)
print('ロジスティック回帰:{0:.3f}'.format(log_accuracy) )

- K近傍法

In [ ]:
# kを変化させてkの値を決定する
# kを1～30まで変化させる
k_range = range(1, 31)

Kaccuracy = []
for k in k_range:
    # ｋのインスタンスの作成
    knn = KNeighborsClassifier(n_neighbors=k)
    # 学習
    knn.fit(X_train, Y_train)
    
    # 'test'データの予測
    k_pred = knn.predict(X_test)
    
    # 予測の評価
    Kaccuracy.append(metrics.accuracy_score(Y_test, k_pred))

In [ ]:
# kの値を変化させたときの正答率をプロットする
plt.plot(k_range, Kaccuracy)

# 軸名の設定
plt.xlabel('K for KNN')
plt.ylabel('Testing Accuracy')

plt.show()

In [ ]:
# 精度が最も高い'k'のindexを探す
kmax = Kaccuracy.index(max(Kaccuracy))
print(kmax)

In [ ]:
# 精度が最も高くなる'k'の値で再度学習とテストを行う
# インスタンスの作成
Knn = KNeighborsClassifier(n_neighbors = kmax+1)

# 学習
Knn.fit(X_train, Y_train)

# 'test'データの予測
knn_predict = Knn.predict(X_test)

# 予測の評価
knn_accuracy = metrics.accuracy_score(Y_test, knn_predict)
print('K近傍法:{:.3f}'.format(knn_accuracy))

- サポートベクターマシン

In [ ]:
# インスタンスの作成
SVC_model = SVC()

# 学習
SVC_model.fit(X_train, Y_train)

# 'test'データの予測
SVC_predict = SVC_model.predict(X_test)

# 予測データの評価
SVC_accuracy = metrics.accuracy_score(Y_test, SVC_predict)
print('サポートベクターマシン:{:.3f}'.format(SVC_accuracy))

- ナイーブベイズ法

In [ ]:
# インスタンスの作成
GNB_model = GaussianNB()

# 学習
GNB_model.fit(X_train, Y_train)

# 'test'データの予測
GNB_predict = GNB_model.predict(X_test)

# 予測の評価
GNB_accuracy = metrics.accuracy_score(Y_test, GNB_predict)
print('GNB:{:.3f}'.format(GNB_accuracy))

- 決定木モデル

In [ ]:
# iを変化させてiの値を決定する
# iを1～101まで変化させる
i_range = range(1, 101)

Iaccuracy=[]

for i in i_range:
    # インスタンスの作成
    tree_model = DecisionTreeClassifier(max_depth=i)
    # 学習
    tree_model.fit(X_train, Y_train)
    # 'test'データの予測
    tree_predict = tree_model.predict(X_test)
    # 予測の評価
    Iaccuracy.append(metrics.accuracy_score(Y_test, tree_predict))

In [ ]:
# 予測の精度の結果をプロットする
plt.plot(i_range, Iaccuracy)

# 軸名の設定
plt.xlabel('i for DTC')
plt.ylabel('Testing Accuracy')

plt.show()

In [ ]:
# 精度が最も高くなるような'i'のindexを探す
imax = Iaccuracy.index(max(Iaccuracy))
print(imax)

In [ ]:
# i=imaxとして再度学習とテスト
# インスタンスの作成
Tree_model = DecisionTreeClassifier(max_depth = imax+1)
# 学習
Tree_model.fit(X_train, Y_train)
# 'test'データの予測
Tree_predict = Tree_model.predict(X_test)
# 予測の評価
Tree_accuracy = metrics.accuracy_score(Y_test, Tree_predict)
print('決定木:{:.3f}'.format(Tree_accuracy))

- ランダムフォレスト

In [ ]:
# インスタンスの作成
clf_model = RandomForestClassifier()
# 学習
clf_model.fit(X_train, Y_train)
# 'test'データの予測
clf_predict = clf_model.predict(X_test)
# 予測の評価
clf_accuracy = metrics.accuracy_score(Y_test, clf_predict)
print('ランダムフォレスト:{:.3f}'.format(clf_accuracy))

- 主成分分析

In [ ]:
# データの標準化
# インスタンスの作成
sc = StandardScaler()
# 説明変数の標準化
df_std = sc.fit_transform(df_data)

In [ ]:
# 'df_std'をDataFrameに変換する
df_std = pd.DataFrame(df_std, columns=df_data.columns.values)
df_std.head()

In [ ]:
# 主成分分析のインスタンスの作成
pca = PCA(n_components=3)
pca.fit(df_std)

In [ ]:
print('各主成分の分散:{}'.format(pca.explained_variance_))

In [ ]:
# 情報の圧縮率
print('各主成分の分散割合:{}'.format(pca.explained_variance_ratio_))

In [ ]:
# 主成分分析を行う
array_pca = pca.transform(df_std)

# DataFrameに変換する
df_pca = pd.DataFrame(array_pca, columns={'first_ingredient', 'second_ingredient', 'third_ingredient'})

# 主成分分析の結果と目的変数を結合する
df_pca1 = pd.concat([df_pca, df_target], axis=1)
df_pca.head()

In [ ]:
# 主成分分析の結果をプロットする
fig = plt.figure(figsize=(8,8))
ax = Axes3D(fig)

# Axesのタイトルを設定
ax.set_title('Pricipal component analysis')

# 軸ラベルを設定
ax.set_xlabel('first ingredient', size=14, color='r')
ax.set_ylabel('second ingredient', size=14, color='r')
ax.set_zlabel('third ingredient', size=14, color='r')

# 'DEATH_EVENT'を並び替え
cluster = sorted(set(df_pca1['DEATH_EVENT']))
# 色の指定
colors = list(mcolors.TABLEAU_COLORS.keys())

# 'DEATH_EVENT'の値ごとにプロットする
for i, x in enumerate(cluster):
    df_pca2 = df_pca1[df_pca1['DEATH_EVENT'] == x]
    X1 = df_pca2['first_ingredient']
    Y1 = df_pca2['second_ingredient']
    Z1 = df_pca2['third_ingredient']
    p = ax.scatter(X1,Y1,Z1, c=colors[i], label=x)

plt.legend()
plt.show()

In [ ]:
# 'train'データと'test'データを分ける
X_train_pca, X_test_pca, Y_train_pca, Y_test_pca = train_test_split(df_pca, df_target, random_state=0)

In [ ]:
# ロジスティック回帰のインスタンスの作成
log_model_pca = LogisticRegression(max_iter=1000)
# 学習
log_model_pca.fit(X_train_pca, Y_train_pca)
# 'test'データの予測
pca_predict = log_model_pca.predict(X_test_pca)
pca_accuracy = metrics.accuracy_score(Y_test_pca, pca_predict)
print('主成分分析:{:.3f}'.format(pca_accuracy))


# 7.モデルの評価

In [ ]:
df_accuracy = pd.DataFrame({'モデル':['ロジスティック回帰', 'K近傍法', 'SVC', 'ナイーブベイズ法','決定木モデル', 'ランダムフォレスト', '主成分分析'],
                           '評価':[log_accuracy, knn_accuracy, SVC_accuracy, GNB_accuracy, Tree_accuracy, clf_accuracy, pca_accuracy]})

In [ ]:
# 予測の精度が高い順にsort
df_accuracy.sort_values(by='評価', ascending=False)

# 8.スタッキング

In [ ]:
# 各モデルの予測の結果一覧
df_predict = pd.DataFrame({'Logistic':log_predict,
                          'KNN':knn_predict,
                          'GNB':GNB_predict,
                          'DTC':Tree_predict,
                          'SVC':SVC_predict,
                          'CLF':clf_predict,
                          'PCA':pca_predict,
                          'real':Y_test['DEATH_EVENT']})

In [ ]:
#  ランダムフォレストとナイーブベイズ法以外のモデルを用いてスタッキング
df_predict['sum'] =  df_predict.Logistic  + df_predict.KNN + df_predict.CLF + df_predict.PCA + df_predict.SVC

In [ ]:
df_predict.head()

In [ ]:
# 各モデルの予測を考慮した結果
df_predict.loc[(df_predict['sum'] >= 0) & (df_predict['sum'] <= 2 ) , 'sum'] = 0
df_predict.loc[(df_predict['sum'] > 2 ), 'sum'] = 1
predict_sum = df_predict['sum']

In [ ]:
# 予測の評価
sum_accuracy = metrics.accuracy_score(Y_test, predict_sum)
print('スタッキングの予測精度:{:.3f}'.format(sum_accuracy))

In [ ]:
# 5つのモデルを用いてスタッキングした結果と決定木モデル，ナイーブベイズ法を用いてスタッキング
df_all = pd.DataFrame({'DTC':Tree_predict,
                       'GNB':GNB_predict,
                       'SUM':predict_sum,
                       'real':Y_test['DEATH_EVENT']})

In [ ]:
df_all['all'] = df_all.DTC + df_all.GNB + df_all.SUM

In [ ]:
# 各モデルの予測を考慮した結果
df_all.loc[(df_all['all'] >= 0) & (df_all['all'] <= 1 ) , 'all'] = 0
df_all.loc[(df_all['all'] > 1 ), 'all'] = 1
predict_all = df_all['all']

In [ ]:
# スタッキングの予測の評価
all_accuracy = metrics.accuracy_score(Y_test, predict_all)
print('スタッキングの予測精度:{:.3f}'.format(all_accuracy))

# 9.まとめ

複数のモデルを用いてスタッキングし予測した結果より，決定木モデルだけで予測した場合の方が精度が高かった．スタッキングの重みの付け方などを工夫してやれば，精度が上がるのではないかと考えられる．また，一般的に心血管疾患の危険因子として知られる喫煙,高血圧，糖尿病の要素を上手く工夫してやれば，さらに精度を高められるのではないかと考えた．